In [1]:
import os, sys
sys.path.insert(0, '/global/homes/j/jwp/DC2/gcr-catalogs')
print(GCRCatalogs.__file__)
import GCRCatalogs
from astropy.coordinates import SkyCoord, matching
import astropy.units as u
%load_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import pandas as pd

### Matching Source and Object catalogs back to truth

Truth matching will only be applied to the objects. This means the sources that did not get matched to objects will not be matched to the truth.

The "truth" is the union of galaxies in `proto-dc2_v3.0` (on which Run 1.2i was based) and `dc2_truth_run1.2_static`. We include the truth catalog for the stars that are not in the extragalactic catalog.

In [24]:
# Fetch protoDC2 with the columns we need, restricted to our area
extragal_gcr = GCRCatalogs.load_catalog('proto-dc2_v3.0_test') # _test skips the md5 check
# 'cosmoDC2_v1.0_image' takes ~14 sec
extragal_quantities = ['galaxy_id', 'ra_true', 'dec_true', 'redshift_true', 
                      'size_bulge_true', 'size_minor_bulge_true', 'sersic_bulge', 'ellipticity_1_bulge_true',
                      'ellipticity_2_bulge_true', 'ellipticity_bulge_true',
                      'size_disk_true', 'size_minor_disk_true', 'sersic_disk', 'ellipticity_1_disk_true',
                      'ellipticity_2_disk_true', 'ellipticity_disk_true',
                      'bulge_to_total_ratio_i',
                      'mag_true_u_lsst',
                      'mag_true_g_lsst',
                      'mag_true_r_lsst',
                      'mag_true_i_lsst',
                      'mag_true_z_lsst',
                      'mag_true_Y_lsst',
                      'halo_mass',]
galaxies_dict = extragal_gcr.get_quantities(protodc2_quantities, filters=[])
galaxies = pd.DataFrame(galaxies_dict, index=None)
print(galaxies.shape)

In [10]:
# Find which ra, dec box corresponds to Tract 4850  
from skymap_utils import get_tract_corners
corners = get_tract_corners(tract_id=4850)

tract 4850 has corners (56.410, -29.783), (54.618, -29.783), (54.631, -28.228), (56.397, -28.228) (RA, Dec deg) and 7 x 7 patches


In [18]:
min_ra, min_dec = np.min(corners, axis=0)
max_ra, max_dec = np.max(corners, axis=0)
buffer = 0.1 # deg

In [49]:
# Fetch truth with the columns we need, restricted to our area
truth_gcr = GCRCatalogs.load_catalog('dc2_truth_run1.2_static')
truth_quantities = ['object_id', 'sprinkled', 'star', 'agn', 'ra', 'dec', 'redshift'] + list('ugrizy') 
#print(sorted(truth_gcr.list_all_quantities(include_native=True)))
truth_filters = ['ra > %f' %(min_ra - buffer),
                'ra < %f' %(max_ra + buffer),
                'dec > %f' %(min_dec - buffer),
                'dec < %f' %(max_dec + buffer),]
truth_dict = truth_gcr.get_quantities(truth_quantities, native_filters=truth_filters)
truth_df = pd.DataFrame(truth_dict, index=None)

### Join extragalactic catalog with truth catalog
In the case of protoDC2, this step is necessary to rotate the protoDC2 field onto the DC2 field. Only galaxies will survive the join.

In [51]:
truth_extended = galaxies.merge(truth_df, how='inner', left_on='galaxy_id', right_on='object_id')

In [55]:
truth_extended.head()

,halo_mass,sersic_bulge,ellipticity_bulge_true,mag_true_u_lsst,ellipticity_disk_true,size_bulge_true,ellipticity_2_bulge_true,mag_true_z_lsst,galaxy_id,size_minor_disk_true,...,y,sprinkled,ra,u,object_id,g,star,redshift,r,agn
0,3.509520e+11,4.0,0.347326,13.402844,0.046173,9.898567,0.052789,10.861640,7,6.225793,...,10.498124,False,54.599659,12.220207,7,11.366055,False,0.007052,10.996866,True
1,1.164449e+11,4.0,0.298637,19.124431,0.010436,2.022574,0.256484,17.642028,26,2.108287,...,17.562220,False,54.913981,19.151460,26,18.259135,False,0.012973,17.925473,True
2,1.940749e+10,4.0,NaN,19.726181,0.105133,0.000000,NaN,17.586020,52,1.940603,...,17.627993,False,54.966244,19.625544,52,18.495268,False,0.013717,17.978307,False
3,3.072852e+10,4.0,0.193061,22.939206,0.137340,1.098874,-0.148197,21.373796,60,0.458977,...,21.272098,False,55.077733,22.931867,60,21.894835,False,0.020161,21.543838,False
4,8.133354e+12,4.0,0.210889,20.949763,0.128405,0.923869,-0.035678,19.838191,80,0.393055,...,19.757425,False,54.657801,21.029238,80,20.132630,False,0.023965,19.917222,True


### Save a subset of the truth catalog with just stars

In [52]:
truth_point = truth_df[truth_df['star']==True].copy()
print(truth_point.shape)

### Concatenate the coordinates of `truth_extended` and `truth_point` in preparation for matching

In [60]:
truth_ra = np.concatenate([truth_extended['ra'].values, truth_point['ra'].values])
truth_dec = np.concatenate([truth_extended['dec'].values, truth_point['dec'].values])
truth_skyCoord = SkyCoord(truth_ra, truth_dec, unit=(u.deg, u.deg))

obj = pd.read_csv('obj_opsim.csv', index_col=None)
obj_skyCoord = SkyCoord(obj['ra'].values, obj['dec'].values, unit=(u.deg, u.deg))

In [89]:
# Use astropy matching as in:
# https://github.com/LSSTDESC/DC2-production/blob/881125e14ff0cd5c4ea35b7725273369167cf98a/scripts/merge_source_cat.py#L292

matching_radius = 1.0 * u.arcsec
idx, sep2d, _ = matching.match_coordinates_sky(obj_skyCoord, truth_skyCoord)

In [111]:
def assign_truth_id(idx, sep2d, truth_extended, truth_point, matching_radius):
    is_fail = sep2d > matching_radius
    n_objects = len(idx)
    matched_truth_ids = np.zeros(n_objects) # Initialize
    # Which objects were matched to extended vs. point?
    n_extended = truth_extended.shape[0]
    is_extended = idx < n_extended
    is_point = idx >= n_extended
    # Get object_id of truth match
    matched_truth_extended = truth_extended.iloc[idx[is_extended]]['object_id'].values.astype(int)
    matched_truth_point = truth_point.iloc[idx[is_point] % n_extended - 1]['object_id'].values.astype(int)
    # Populate object_id for extended, point
    matched_truth_ids[is_extended] = matched_truth_extended
    matched_truth_ids[is_point] = matched_truth_point
    # Flag match failure
    matched_truth_ids[is_fail] = -1

    return matched_truth_ids

truth_ids = assign_truth_id(idx, sep2d, truth_extended, truth_point, matching_radius)

In [115]:
obj['truth_id'] = truth_ids

In [116]:
obj.to_csv('matched_obj_opsim.csv', index=None)